In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.load('./X_new.npy')
Y = np.load('./Y_new2.npy')

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
Xpd = pd.DataFrame(X)
Ypd = pd.DataFrame(Y)

In [5]:
Ypd[0].value_counts()

3    1271
5    1271
1    1271
4    1271
0    1271
2    1087
Name: 0, dtype: int64

In [6]:
from imblearn.over_sampling import SMOTE
sm = SMOTE('minority')
X_rs, Y_rs = sm.fit_sample(Xpd, Ypd)

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:638: FutureWarning: Pass sampling_strategy=minority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


In [10]:
Y_rs[0].value_counts()

3    1271
2    1271
5    1271
1    1271
4    1271
0    1271
Name: 0, dtype: int64

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_rs, Y_rs, test_size=0.33, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5109, 180)
(5109, 1)
(2517, 180)
(2517, 1)


In [13]:
scaler = StandardScaler()
# Fit on training set only.

scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_tr = scaler.transform(X_train)
X_te = scaler.transform(X_test)

In [14]:
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(X_tr,y_train)
print(pca.n_components_)

83


In [15]:
X_tr_pca = pca.transform(X_tr)
X_te_pca = pca.transform(X_te)

In [16]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

In [21]:
bagging = BaggingClassifier(RandomForestClassifier(n_estimators=1000), max_samples=0.5, max_features=0.5)

In [22]:
bagging.fit(X_tr_pca,y_train)

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=1000),
                  max_features=0.5, max_samples=0.5)

In [28]:
clf = RandomForestClassifier(n_estimators = 1000, random_state = 42, max_features = "auto", min_samples_leaf = 50)
clf.fit(X_tr_pca,y_train)

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(min_samples_leaf=50, n_estimators=1000, random_state=42)

In [23]:
print("Train Score ----> ",bagging.score(X_tr_pca,y_train))
print("Train Score ----> ", bagging.score(X_te_pca,y_test))

Train Score ---->  0.9479350166373067
Train Score ---->  0.44656336909018673


In [29]:
print("Train Score ----> ",clf.score(X_tr_pca,y_train))
print("Train Score ----> ", clf.score(X_te_pca,y_test))

Train Score ---->  0.5055783910745743
Train Score ---->  0.41080651569328563


In [52]:
from tqdm import tqdm
for i in [100,500,1000,2000,3000]:
    clf = RandomForestClassifier(n_estimators = i, random_state = 42, max_features = "sqrt", min_samples_leaf = 5)
    clf.fit(X_tr_pca,y_train[0])
    print('-------------- ',i,' --------------')
    print("Train Score ----> ",clf.score(X_tr_pca,y_train[0]))
    print("Test Score ----> ", clf.score(X_te_pca,y_test[0]))

--------------  100  --------------
Train Score ---->  0.9655509884517518
Test Score ---->  0.45172824791418353
--------------  500  --------------
Train Score ---->  0.9688784497944803
Test Score ---->  0.46523639253079063
--------------  1000  --------------
Train Score ---->  0.9700528479154433
Test Score ---->  0.46444179578863726
--------------  2000  --------------
Train Score ---->  0.9700528479154433
Test Score ---->  0.46642828764402067
--------------  3000  --------------
Train Score ---->  0.9694656488549618
Test Score ---->  0.4676201827572507


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold, cross_validate

In [58]:
model=MLPClassifier(alpha=0.01, batch_size=64, epsilon=1e-08,
                    hidden_layer_sizes=(300,200,50), learning_rate='adaptive', max_iter=2000, 
                    activation='relu',learning_rate_init=0.001, verbose=True, early_stopping=True)


In [59]:
cvresults=cross_validate(model, X_tr_pca, y_train, cv=5, return_train_score=True)

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 1.61845055
Validation score: 0.422983
Iteration 2, loss = 1.34423382
Validation score: 0.452323
Iteration 3, loss = 1.25509685
Validation score: 0.430318
Iteration 4, loss = 1.16422795
Validation score: 0.484108
Iteration 5, loss = 1.08186958
Validation score: 0.488998
Iteration 6, loss = 1.00894175
Validation score: 0.493888
Iteration 7, loss = 0.94479430
Validation score: 0.459658
Iteration 8, loss = 0.87104597
Validation score: 0.476773
Iteration 9, loss = 0.79209105
Validation score: 0.508557
Iteration 10, loss = 0.73051394
Validation score: 0.537897
Iteration 11, loss = 0.66675968
Validation score: 0.511002
Iteration 12, loss = 0.59307523
Validation score: 0.503667
Iteration 13, loss = 0.53418691
Validation score: 0.481663
Iteration 14, loss = 0.48663017
Validation score: 0.493888
Iteration 15, loss = 0.45631780
Validation score: 0.511002
Iteration 16, loss = 0.39146492
Validation score: 0.508557
Iteration 17, loss = 0.35038338
Validation score: 0.491443
Iterat

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 1.56656496
Validation score: 0.400978
Iteration 2, loss = 1.34358449
Validation score: 0.440098
Iteration 3, loss = 1.24747093
Validation score: 0.454768
Iteration 4, loss = 1.15229429
Validation score: 0.459658
Iteration 5, loss = 1.06871253
Validation score: 0.484108
Iteration 6, loss = 1.00707970
Validation score: 0.452323
Iteration 7, loss = 0.92064155
Validation score: 0.474328
Iteration 8, loss = 0.84980149
Validation score: 0.464548
Iteration 9, loss = 0.77168945
Validation score: 0.474328
Iteration 10, loss = 0.69309113
Validation score: 0.440098
Iteration 11, loss = 0.62099808
Validation score: 0.459658
Iteration 12, loss = 0.57203763
Validation score: 0.481663
Iteration 13, loss = 0.50716385
Validation score: 0.466993
Iteration 14, loss = 0.45623578
Validation score: 0.464548
Iteration 15, loss = 0.43457215
Validation score: 0.469438
Iteration 16, loss = 0.36436125
Validation score: 0.462103
Validation score did not improve more than tol=0.000100 for 10 co

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 1.57279150
Validation score: 0.408313
Iteration 2, loss = 1.32453071
Validation score: 0.396088
Iteration 3, loss = 1.23842247
Validation score: 0.435208
Iteration 4, loss = 1.13449286
Validation score: 0.447433
Iteration 5, loss = 1.05249951
Validation score: 0.418093
Iteration 6, loss = 0.97418191
Validation score: 0.452323
Iteration 7, loss = 0.88998903
Validation score: 0.459658
Iteration 8, loss = 0.82229522
Validation score: 0.484108
Iteration 9, loss = 0.74575559
Validation score: 0.444988
Iteration 10, loss = 0.67273381
Validation score: 0.464548
Iteration 11, loss = 0.62539918
Validation score: 0.462103
Iteration 12, loss = 0.55023477
Validation score: 0.469438
Iteration 13, loss = 0.46563983
Validation score: 0.452323
Iteration 14, loss = 0.40833441
Validation score: 0.464548
Iteration 15, loss = 0.35837964
Validation score: 0.466993
Iteration 16, loss = 0.31300432
Validation score: 0.462103
Iteration 17, loss = 0.28922403
Validation score: 0.459658
Iterat

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 1.56544587
Validation score: 0.422983
Iteration 2, loss = 1.34729620
Validation score: 0.457213
Iteration 3, loss = 1.24557162
Validation score: 0.464548
Iteration 4, loss = 1.15399976
Validation score: 0.476773
Iteration 5, loss = 1.08454391
Validation score: 0.496333
Iteration 6, loss = 0.98259410
Validation score: 0.476773
Iteration 7, loss = 0.91947656
Validation score: 0.488998
Iteration 8, loss = 0.83587086
Validation score: 0.481663
Iteration 9, loss = 0.75598094
Validation score: 0.503667
Iteration 10, loss = 0.69183885
Validation score: 0.508557
Iteration 11, loss = 0.63184066
Validation score: 0.498778
Iteration 12, loss = 0.59550296
Validation score: 0.488998
Iteration 13, loss = 0.51367969
Validation score: 0.471883
Iteration 14, loss = 0.43901998
Validation score: 0.503667
Iteration 15, loss = 0.39429273
Validation score: 0.486553
Iteration 16, loss = 0.34676058
Validation score: 0.511002
Iteration 17, loss = 0.32706179
Validation score: 0.479218
Iterat

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 1.57030803
Validation score: 0.405868
Iteration 2, loss = 1.32881919
Validation score: 0.462103
Iteration 3, loss = 1.20625328
Validation score: 0.484108
Iteration 4, loss = 1.11375382
Validation score: 0.498778
Iteration 5, loss = 1.03224396
Validation score: 0.496333
Iteration 6, loss = 0.94888049
Validation score: 0.479218
Iteration 7, loss = 0.88521995
Validation score: 0.491443
Iteration 8, loss = 0.81358142
Validation score: 0.498778
Iteration 9, loss = 0.74516968
Validation score: 0.493888
Iteration 10, loss = 0.68367503
Validation score: 0.476773
Iteration 11, loss = 0.60966549
Validation score: 0.506112
Iteration 12, loss = 0.55353336
Validation score: 0.486553
Iteration 13, loss = 0.49498423
Validation score: 0.471883
Iteration 14, loss = 0.42980870
Validation score: 0.464548
Iteration 15, loss = 0.38861282
Validation score: 0.476773
Iteration 16, loss = 0.33162407
Validation score: 0.491443
Iteration 17, loss = 0.28796454
Validation score: 0.486553
Iterat

In [60]:
cvresults

{'fit_time': array([4.81967783, 3.57555199, 4.73638201, 7.96942091, 7.42852783]),
 'score_time': array([0.00998092, 0.01314402, 0.00914693, 0.01072001, 0.00849414]),
 'test_score': array([0.50587084, 0.48043053, 0.49412916, 0.53033268, 0.46914789]),
 'train_score': array([0.78272572, 0.64472718, 0.75189626, 0.93956447, 0.94080235])}

In [62]:
model.fit(X_tr_pca,y_train)
model.score(X_te_pca,y_test)

/Users/tanaysinghania/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 1.54860165
Validation score: 0.410959
Iteration 2, loss = 1.32139209
Validation score: 0.446184
Iteration 3, loss = 1.22800931
Validation score: 0.455969
Iteration 4, loss = 1.13207585
Validation score: 0.446184
Iteration 5, loss = 1.06693979
Validation score: 0.473581
Iteration 6, loss = 0.97927367
Validation score: 0.469667
Iteration 7, loss = 0.92367531
Validation score: 0.493151
Iteration 8, loss = 0.83873374
Validation score: 0.455969
Iteration 9, loss = 0.77786037
Validation score: 0.465753
Iteration 10, loss = 0.70227981
Validation score: 0.473581
Iteration 11, loss = 0.65616930
Validation score: 0.436399
Iteration 12, loss = 0.58532727
Validation score: 0.440313
Iteration 13, loss = 0.53033834
Validation score: 0.461840
Iteration 14, loss = 0.49750726
Validation score: 0.467710
Iteration 15, loss = 0.42787643
Validation score: 0.465753
Iteration 16, loss = 0.38258805
Validation score: 0.461840
Iteration 17, loss = 0.34218260
Validation score: 0.459883
Iterat

0.5017878426698451